In [1]:
import pickle
from db import get_spectra_sqlite, save_spectra_sqlite, rand_sub_sqlite, rand_sub_sqlite1, \
                      get_mz_ranges, convert_to_dense, plot_mz_hist, filter_spectra
from da import data_augmentation_1, data_augmentation_2
from snn import plot_loss_accuracy, check_pSCNN, build_pSCNN, load_pSCNN, \
                      predict_pSCNN, evaluate_pSCNN
from AutoRes import AutoRes, output_msp
from NetCDF import netcdf_reader
import pandas as pd 

In [2]:
model_name1 = 'model/pSCNN1'
model_name2 = 'model/pSCNN2'
maxn1 = 1
maxn2 = 3
mz_range = (1, 1000)

In [3]:
#train pSCNN1 model
if check_pSCNN(model_name1):
        model1 = load_pSCNN(model_name1)
else:
    para = {'dbname': 'dataset/NIST_Spec0-236200.db',
            'mz_range': mz_range, 
            'aug_num': 200000,
            'noise_level': 0.001,
            'maxn': maxn1,
            'layer_num': 3,
            'batch': 200,
            'epoch': 200,
            'lr': 0.001,
            'factor':0.8,
            'min_lr':0.000002,
            'model_name': model_name1}
    model1 = build_pSCNN(para)
#test pSCNN1 model
dbname1 = 'dataset/NIST_Spec-10000.db'
spectra1 = get_spectra_sqlite(dbname1)
convert_to_dense(spectra1, mz_range)  
aug_eval1 = data_augmentation_1(spectra1, 100000, maxn1, 0.001)
eval_acc1 = evaluate_pSCNN(model1, [aug_eval1['R'], aug_eval1['S']], aug_eval1['y'])
yp1 = predict_pSCNN(model1, [aug_eval1['R'], aug_eval1['S']])

Data augmentation: 100%|█████████████████████████████████████████████████████| 100000/100000 [00:15<00:00, 6573.23it/s]


6250/6250 [==============================] - 29s 4ms/step - loss: 6.4092e-04 - accuracy: 0.9998


In [4]:
#train pSCNN2 model
with open('dataset/data.pk', 'rb') as file_1:
     c = pickle.load(file_1)  
if check_pSCNN(model_name2):
    model2 = load_pSCNN(model_name2)
else:
    para = {'dbname': 'dataset/NIST_Spec0-236200.db',
            'mz_range': mz_range,
            'aug_num0': 100000,
            'aug_num1': 100000,
            'noise_level': 0.001,
            'maxn': maxn2,
            'layer_num': 2,
            'batch': 200,
            'epoch': 200,
            'lr': 0.0001,
            'factor':0.8,
            'min_lr':0.0000001,
            'c':c,
            'model_name': model_name2}
    model2 = build_pSCNN(para)
#test pSCNN2 model
with open('dataset/data1.pk', 'rb') as file_1:
     c1 = pickle.load(file_1) 
spectra2 = get_spectra_sqlite('dataset/NIST_Spec-10000.db')
convert_to_dense(spectra2, mz_range)
aug_eval2 = data_augmentation_2(spectra2, c1, 10000, 90000, maxn2, 0.001)
eval_acc2 = evaluate_pSCNN(model2, [aug_eval2['R'], aug_eval2['S']], aug_eval2['y'])
yp2 = predict_pSCNN(model2, [aug_eval2['R'], aug_eval2['S']])

Data augmentation: 100%|███████████████████████████████████████████████████████| 90000/90000 [00:14<00:00, 6118.36it/s]


6250/6250 [==============================] - 20s 3ms/step - loss: 0.2072 - accuracy: 0.9250


In [5]:
#test AutoRes
filename = ['data/1-0.2-3.CDF']
msp = ['msp/1-0.2.MSP']
csv = ['results/1-0.2.csv']
for i in range(len(filename)):
    ncr = netcdf_reader(filename[i], True)
    sta_S, area, rt, R2 = AutoRes(ncr, model1, model2)
    output_msp(msp[i], sta_S, rt)
    df = pd.DataFrame({'rt': rt, 'area': area, 'R2': R2})
    df.to_csv(csv[0], index=False)